# Building a Fraud Prediction Model with EvalML

In this demo, we will build an optimized fraud prediction model using EvalML. To optimize the model we will set up an objective function based on some assumptions about our business. 

In [1]:
%load_ext autoreload
%autoreload 2
import evalml
from evalml.objectives import FraudDetection
from evalml.preprocessing import split_data

/Users/kanter/.pyenv/versions/3.6.8/envs/evalml/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Load Transactions

In [2]:
X, y = evalml.demos.load_fraud()

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 99992

Labels
False    84.82%
True     15.18%
Name: fraud, dtype: object


## Define Objective

In [3]:
fraud_objective = FraudDetection(
    retry_percentage=.5,
    interchange_fee=.02,
    fraud_payout_percentage=.75,
    amount_col='amount'
)

## Search for best modeling pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

In [4]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']
X = X.select_dtypes(include=numerics)
X_train, X_holdout, y_train, y_holdout = split_data(X, y, test_size=.8, random_state=0)

Because the fraud labels are binary, we will use `AutoClassifier`. When we call `.fit()`, the search for the best pipeline will begin. 

In [ ]:
clf = evalml.AutoClassifier(objective=fraud_objective,
                            max_pipelines=10)

clf.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Fraud Detection. Lower score is better.

Searching up to 10 pipelines.

Possible model types: linear_model, random_forest, xgboost

Found array with 0 feature(s) (shape=(10665, 0)) while a minimum of 1 is required.      
Found array with 0 feature(s) (shape=(10665, 0)) while a minimum of 1 is required.      
Found array with 0 feature(s) (shape=(10665, 0)) while a minimum of 1 is required.      
Testing Random Forest w/ Imputation:  60%|██████    | 6/10 [01:07<00:44, 11.00s/it]     

/Users/kanter/.pyenv/versions/3.6.8/envs/evalml/lib/python3.6/site-packages/sklearn/feature_selection/base.py:79: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  UserWarning)
/Users/kanter/.pyenv/versions/3.6.8/envs/evalml/lib/python3.6/site-packages/sklearn/feature_selection/base.py:79: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  UserWarning)
/Users/kanter/.pyenv/versions/3.6.8/envs/evalml/lib/python3.6/site-packages/sklearn/feature_selection/base.py:79: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  UserWarning)


### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the objective function we defined

In [ ]:
clf.rankings

to select the best pipeline we can run

In [ ]:
pipeline = clf.best_pipeline

to select another pipeline we can use the id

In [ ]:
pipeline = clf.get_pipeline(0)

### Describe pipeline

You can get more details about any pipeline. Including 

In [ ]:
clf.describe_pipeline(1)